# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 3.2.1
- **Date :** 09/11/2023
- **Heure :** 14:56
- **Données :** `ready_Evaporation_Sunshine_Cloud9am_Cloud3pm.csv`
- **Tri :** par défaut
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** TSF

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../../data/processed/omar/ready_Evaporation_Sunshine_Cloud9am_Cloud3pm.csv", index_col = 1)
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,year,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-01,13.3,34.2,0.0,61.0,11.0,14.013296,67.0,51.422456,1005.6,1015.640417,...,2009,1,1,1,3.141593,1.178097,2.917968,-33.881667,150.744163,0
2009-01-02,14.7,26.1,0.0,46.0,7.0,24.000000,59.0,54.000000,1012.9,1013.500000,...,2009,1,2,1,5.497787,5.497787,5.497787,-33.881667,150.744163,0
2009-01-03,13.6,22.3,0.0,30.0,6.0,15.000000,57.0,51.000000,1021.9,1019.200000,...,2009,1,3,1,1.178097,5.890486,0.785398,-33.881667,150.744163,0
2009-01-04,17.7,31.2,0.0,39.0,9.0,15.000000,62.0,43.000000,1018.7,1013.600000,...,2009,1,4,1,0.785398,1.178097,1.570796,-33.881667,150.744163,0
2009-01-05,15.5,38.8,0.0,50.0,7.0,17.000000,67.0,19.000000,1013.2,1007.600000,...,2009,1,5,1,3.926991,1.178097,3.141593,-33.881667,150.744163,0


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18907 entries, 2009-01-01 to 2017-06-25
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         18907 non-null  float64
 1   MaxTemp         18907 non-null  float64
 2   Rainfall        18907 non-null  float64
 3   WindGustSpeed   18907 non-null  float64
 4   WindSpeed9am    18907 non-null  float64
 5   WindSpeed3pm    18907 non-null  float64
 6   Humidity9am     18907 non-null  float64
 7   Humidity3pm     18907 non-null  float64
 8   Pressure9am     18907 non-null  float64
 9   Pressure3pm     18907 non-null  float64
 10  Temp9am         18907 non-null  float64
 11  Temp3pm         18907 non-null  float64
 12  RainToday       18907 non-null  int64  
 13  RainTomorrow    18907 non-null  int64  
 14  year            18907 non-null  int64  
 15  month           18907 non-null  int64  
 16  day             18907 non-null  int64  
 17  LocationNum     18907 

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Instanciation d'un modèle classificateur via un Pipeline

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.interval_based import TimeSeriesForestClassifier

steps = [
    ("concatenate", ColumnConcatenator()), ## Cette étape permet de transformer les jeux de données multivariées en univariées afin qu'ils puissent être traités par `TimeSeriesForestClassifier`.
    ("classify", TimeSeriesForestClassifier()),
]

clf_rf_ts = Pipeline(steps)

In [10]:
# Récupération des paramètres initiaux de la TSF

clf_rf_ts["classify"].get_params()

{'min_interval': 3, 'n_estimators': 200, 'n_jobs': 1, 'random_state': None}

In [11]:
# Entraînement du modèle

clf_rf_ts.fit(X_train, y_train)

Pipeline(steps=[('concatenate', ColumnConcatenator()),
                ('classify', TimeSeriesForestClassifier())])

In [12]:
# Récupération des paramètres ajustés de la TSF

clf_rf_ts["classify"].get_fitted_params()

{'classes': array([0, 1]),
 'intervals': [array([[15, 18],
         [ 3, 16],
         [ 4, 19],
         [ 1,  4]]),
  array([[ 4, 10],
         [14, 17],
         [ 5, 15],
         [ 1,  6]]),
  array([[16, 19],
         [ 1, 14],
         [11, 14],
         [14, 21]]),
  array([[ 7, 13],
         [17, 20],
         [ 2, 19],
         [ 4,  9]]),
  array([[ 0,  5],
         [17, 20],
         [14, 19],
         [ 9, 18]]),
  array([[17, 20],
         [ 4, 19],
         [ 0,  3],
         [18, 21]]),
  array([[10, 16],
         [ 2, 11],
         [ 7, 15],
         [16, 21]]),
  array([[15, 20],
         [14, 17],
         [11, 14],
         [10, 13]]),
  array([[ 2, 21],
         [ 4, 13],
         [ 2, 17],
         [14, 19]]),
  array([[ 5, 14],
         [14, 17],
         [11, 20],
         [15, 18]]),
  array([[12, 15],
         [ 7, 10],
         [18, 21],
         [ 3, 12]]),
  array([[14, 21],
         [ 2, 17],
         [19, 22],
         [ 6, 19]]),
  array([[16, 19],
     

In [13]:
# Réalisation des prédictions

y_pred = clf_rf_ts.predict(X_test)

## 5. Évaluation

In [14]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,1882,206
1,356,707


In [15]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      2088
           1       0.77      0.67      0.72      1063

    accuracy                           0.82      3151
   macro avg       0.81      0.78      0.79      3151
weighted avg       0.82      0.82      0.82      3151

